In [1]:
import os
import pandas as pd
import numpy as np
import math
import ast

In [2]:
def calculate_app_add_cost(dict1, buy, sell):
    num_stocks = len(dict1)
    total_turnover = buy+sell
    extra_cost = (16*num_stocks) + (math.ceil(total_turnover/(num_stocks*1000))*num_stocks)
    return math.ceil(extra_cost)

In [3]:
def add_columns_for_extra_charges(df):
    
    portfolio_list = list(df['portfolio'])
    
    selling_amount_list = list(df['investment_value'])
    buying_amount_list = [100000.00] + selling_amount_list[:-1]
    extra_cost_list = []

    for i in range(len(portfolio_list)):
        portfolio_dict = ast.literal_eval(portfolio_list[i])
        buy = buying_amount_list[i]
        sell = selling_amount_list[i]
        extra_cost = calculate_app_add_cost(portfolio_dict, buy, sell)
        extra_cost_list.append(extra_cost)
    
    return_list = []
    for i in range(len(portfolio_list)):
        returns = (selling_amount_list[i]/buying_amount_list[i])
        return_list.append(returns)

    new_selling_list = []

    for i in range(len(portfolio_list)):
        if i == 0:
            new_sell = selling_amount_list[0] - extra_cost_list[0]
            new_selling_list.append(round(new_sell,2))
        else:
            new_sell = (new_selling_list[i-1]*return_list[i]) - extra_cost_list[i]
            new_selling_list.append(round(new_sell,2))
    
    df['after rebalancing cost'] = new_selling_list
    
    return df

In [4]:
def process_file_with_initial_investment(filepath):
    # Extract strategy ids from filename
    filename = os.path.basename(filepath)
    stock_selection_id, weight_allocation_id = map(int, filename.replace('strategy_', '').replace('.csv', '').split('_'))

    # Read CSV file
    df = pd.read_csv(filepath)

    df = add_columns_for_extra_charges(df)

    df.to_csv(filepath, index=False)
    
    # Initial investment amount
    initial_investment = 100000  # 1,00,000 INR

    # Calculate total return
    # final_investment = df['investment_value'].iloc[-1]
    final_investment = df['after rebalancing cost'].iloc[-1]

    total_return = round(final_investment/initial_investment,2)
    # total_return = round(np.log(final_investment/initial_investment),2)
    
    # Calculate individual period returns
    
    # df['previous_investment_value'] = df['investment_value'].shift(1).fillna(initial_investment)
    df['previous_investment_value'] = df['after rebalancing cost'].shift(1).fillna(initial_investment)

    # df['period_return'] = df['investment_value']/df['previous_investment_value']-1
    df['period_return'] = df['after rebalancing cost']/df['previous_investment_value']-1
    # df['period_return'] = np.log(df['after rebalancing cost']/df['previous_investment_value'])-1

    # Count positive and negative returns
    positive_returns = df[df['period_return'] > 0]
    negative_returns = df[df['period_return'] < 0]
    positive_return_count = len(positive_returns)
    negative_return_count = len(negative_returns)

    # Sum of positive and negative returns
    sum_positive_return = round(positive_returns['period_return'].sum(),2)
    sum_negative_return = round(negative_returns['period_return'].sum(),2)

    # Success rate calculation
    total_periods = len(df)
    success_rate = round(positive_return_count / total_periods if total_periods > 0 else 0,2)

    # period_returns = df['investment_value'] / df['previous_investment_value'] - 1
    period_returns = df['after rebalancing cost']/df['previous_investment_value'] - 1
    # period_returns = np.log(df['after rebalancing cost']/df['previous_investment_value']) - 1
    
    if stock_selection_id in [1,2,5,6,9,10,13,14]:
        cagr_4y = round(np.prod(1 + period_returns) ** (4 / total_periods) - 1,2)
        cagr_3y = round(np.prod(1 + period_returns[4:]) ** (4 / (total_periods-4)) - 1,2)
        cagr_2y = round(np.prod(1 + period_returns[8:]) ** (4 / (total_periods-8)) - 1,2)
        cagr_1y = round(np.prod(1 + period_returns[12:]) ** (4 / (total_periods-12)) - 1,2)
        variance_4y = round(df['period_return'].var(),4)
        variance_3y = round(df['period_return'].iloc[4:].var(),4)
        variance_2y = round(df['period_return'].iloc[8:].var(),4)
        variance_1y = round(df['period_return'].iloc[12:].var(),4)

    else:
        cagr_4y = round(np.prod(1 + period_returns) ** (12 / total_periods) - 1,2)
        cagr_3y = round(np.prod(1 + period_returns[12:]) ** (12 / (total_periods-12)) - 1,2)
        cagr_2y = round(np.prod(1 + period_returns[24:]) ** (12 / (total_periods-24)) - 1,2)
        cagr_1y = round(np.prod(1 + period_returns[36:]) ** (12 / (total_periods-36)) - 1,2)
        var_df = df[['buy_date','period_return']]
        var_df.set_index('buy_date', inplace=True)
        var_df.index = pd.to_datetime(var_df.index)
        quarterly_returns = var_df.resample('Q').apply(lambda x: (1 + x).prod() - 1)
        variance_4y = round(quarterly_returns.var()[0],4)
        variance_3y = round(quarterly_returns[4:].var()[0],4)
        variance_2y = round(quarterly_returns[8:].var()[0],4)
        variance_1y = round(quarterly_returns[12:].var()[0],4)

    return {
        'Strategy_Combination': f'{stock_selection_id}_{weight_allocation_id}',
        'Stock_Selection_Method': stock_selection_id,
        'Weight_Allocation_Method': weight_allocation_id,
        'Total_Return': total_return,
        'CAGR_4Y': cagr_4y,
        'CAGR_3Y': cagr_3y,
        'CAGR_2Y': cagr_2y,
        'CAGR_1Y': cagr_1y,
        'Positive_Return_Count': positive_return_count,
        'Negative_Return_Count': negative_return_count,
        'Sum_Positive_Return': sum_positive_return,
        'Sum_Negative_Return': sum_negative_return,
        'Variance_4Y': variance_4y,
        'Variance_3Y': variance_3y,
        'Variance_2Y': variance_2y,
        'Variance_1Y': variance_1y,
        'Success_Rate': success_rate
    }, df

In [5]:
def print_max_ret_min_var_str(years, highest_success, final_summary):

    # final summary of all together
    final_summary.loc[final_summary['Stock_Selection_Method'].isin([1, 2, 5, 6, 9, 10, 13, 14]), 'Rebalance'] = '3m'
    final_summary.loc[~final_summary['Stock_Selection_Method'].isin([1, 2, 5, 6, 9, 10, 13, 14]), 'Rebalance'] = '1m'

    if highest_success == True:
        max_succ_rate = max(list(final_summary['Success_Rate']))
        final_summary = final_summary[final_summary['Success_Rate']==max_succ_rate]

    # maximum return strategy
    max_ann_ret = max(list(final_summary[f'CAGR_{years}Y']))
    max_ann_ret_df = final_summary[final_summary[f'CAGR_{years}Y']==max_ann_ret]

    # minimum variance strategy
    min_var = min(list(final_summary[f'Variance_{years}Y']))
    min_var_df = final_summary[final_summary[f'Variance_{years}Y']==min_var]

    max_ann_ret_df = max_ann_ret_df.rename(columns={'Strategy_Combination':'Strategy','Variance_4Y':'Var_4Y', 'Variance_1Y':'Var_1Y', 'Backtest_Version':'BT_V', 'Success_Rate':'Success_%'})
    min_var_df = min_var_df.rename(columns={'Strategy_Combination':'Strategy','Variance_4Y':'Var_4Y', 'Variance_1Y':'Var_1Y', 'Backtest_Version':'BT_V', 'Success_Rate':'Success_%'})

    print("*"*10,'\n')
    if highest_success:
        print(f'--> Highest Success Rate & Maximum Return Strategy ({years}Y)\n')
    else:
        print(f'--> Maximum Return Strategy ({years}Y)\n')
    print(max_ann_ret_df[['Strategy','Success_%','Var_4Y','Var_1Y','CAGR_4Y','CAGR_1Y','BT_V','Rebalance']].reset_index(drop=True))
    print('')
    if highest_success:
        print(f'--> Highest Success Rate & Minimum Variance Strategy ({years}Y)\n')
    else:
        print(f'--> Minimum Variance Strategy ({years}Y)\n')
    print(min_var_df[['Strategy','Success_%','Var_4Y','Var_1Y','CAGR_4Y','CAGR_1Y','BT_V','Rebalance']].reset_index(drop=True))
    print('')

In [6]:
# Directory containing the files and the list of directories
directories = []
for i in range(1,11):
    directory = f'../backtesting_results/version_{i}'
    directories.append(directory)

summary_df_list = []

for directory in directories:

    # Initializing an empty DataFrame to store the results
    summary_df = pd.DataFrame()

    # Process each file in the directory
    for filename in os.listdir(directory):
        if filename.startswith("strategy_") and filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            summary_row, df = process_file_with_initial_investment(filepath)
            summary_df = pd.concat([summary_df, pd.DataFrame([summary_row])], ignore_index=True)

    print(len(summary_df))
    summary_df_list.append(summary_df)

    print('\nAvg. CAGR 4Y: ' + str(round(summary_df[['CAGR_4Y','CAGR_1Y', 'Success_Rate']].mean().values[0],2)))
    print('Avg. CAGR 1Y: ' + str(round(summary_df[['CAGR_4Y','CAGR_1Y', 'Success_Rate']].mean().values[1],2)))
    print('Avg. Success Rate: ' + str(round(summary_df[['CAGR_4Y','CAGR_1Y', 'Success_Rate']].mean().values[2],2)) + '\n')

176

Avg. CAGR 4Y: 0.55
Avg. CAGR 1Y: 1.15
Avg. Success Rate: 0.68

176

Avg. CAGR 4Y: 0.51
Avg. CAGR 1Y: 0.91
Avg. Success Rate: 0.69

176

Avg. CAGR 4Y: 0.67
Avg. CAGR 1Y: 1.3
Avg. Success Rate: 0.73

176

Avg. CAGR 4Y: 0.56
Avg. CAGR 1Y: 0.75
Avg. Success Rate: 0.68

176

Avg. CAGR 4Y: 0.53
Avg. CAGR 1Y: 0.72
Avg. Success Rate: 0.68

176

Avg. CAGR 4Y: 0.56
Avg. CAGR 1Y: 0.95
Avg. Success Rate: 0.68

176

Avg. CAGR 4Y: 0.5
Avg. CAGR 1Y: 0.81
Avg. Success Rate: 0.68

176

Avg. CAGR 4Y: 0.5
Avg. CAGR 1Y: 0.54
Avg. Success Rate: 0.65

176

Avg. CAGR 4Y: 0.45
Avg. CAGR 1Y: 0.64
Avg. Success Rate: 0.66

176

Avg. CAGR 4Y: 0.6
Avg. CAGR 1Y: 0.61
Avg. Success Rate: 0.65



In [7]:
final_summary = pd.DataFrame()

for i in range(len(summary_df_list)):
    summary_df = summary_df_list[i]
    summary_df['Backtest_Version'] = i + 1
    final_summary = pd.concat([final_summary, summary_df])

print_max_ret_min_var_str(1,True, final_summary)
print_max_ret_min_var_str(1,False, final_summary)
print_max_ret_min_var_str(4,True, final_summary)
print_max_ret_min_var_str(4,False, final_summary)

final_summary.to_csv('../data/final_backtesting_summary.csv')

********** 

--> Highest Success Rate & Maximum Return Strategy (1Y)

  Strategy  Success_%  Var_4Y  Var_1Y  CAGR_4Y  CAGR_1Y  BT_V Rebalance
0     13_7       0.88  0.0452    0.02     0.76     1.28     5        3m
1     13_6       0.88  0.0452    0.02     0.76     1.28     5        3m

--> Highest Success Rate & Minimum Variance Strategy (1Y)

  Strategy  Success_%  Var_4Y  Var_1Y  CAGR_4Y  CAGR_1Y  BT_V Rebalance
0     10_9       0.88  0.0503  0.0058     0.39     0.71     5        3m

********** 

--> Maximum Return Strategy (1Y)

  Strategy  Success_%  Var_4Y  Var_1Y  CAGR_4Y  CAGR_1Y  BT_V Rebalance
0     9_11       0.81  0.0408  0.0408     0.84     2.14     3        3m
1     5_11       0.81  0.0408  0.0408     0.84     2.14     3        3m
2     1_11       0.81  0.0408  0.0408     0.84     2.14     3        3m

--> Minimum Variance Strategy (1Y)

  Strategy  Success_%  Var_4Y  Var_1Y  CAGR_4Y  CAGR_1Y  BT_V Rebalance
0     13_9       0.69   3.385  0.0034     0.93     0.93    10    